# 1) Package Import

In [1]:
# Importing
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
from bs4 import BeautifulSoup as bs
import re as re
import time
import pandas as pd

In [3]:
import random

# 2) Variable declartations (global)

In [4]:
# webdriver options: tbc if necessary
options = Options()
options.add_argument("start-maximized")

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [1]:
USERNAME = input("Enter the username: ")
PASSWORD = input("Enter the password: ")

print(USERNAME)
print(PASSWORD)

Enter the username:  username
Enter the password:  password


username
password


In [7]:
###############################################################
# keywords4search = ['Aktuar']
# keywords4search = ['Python', 'Data']
# keywords4search = ['Finance', 'Director', 'Data']
# keywords4search = ['Finance', 'Director']
# keywords4search = ['VP Finance']
# keywords4search = ['CFO']
# keywords4search = ['Director FP&A']
keywords4search = ['Director Finance']

###############################################################

In [8]:
#############################################################
# Use manual webbrowsing to find out the correct format used
# by LinkedIn for your location / geography, as this may change 
# with the administrative structure of your country
#############################################################
s_search_loc = 'Deutschland%2C%20Österreich%20und%20die%20Schweiz'
# 'Deutschland'
# 'Düsseldorf%2C%20Nordrhein-Westfalen%2C%20Deutschland'
# 'Nordrhein-Westfalen%2C%20Deutschland'
##############################################################

In [9]:
##############################################################
s_search_distance = '15'     # search distance around specified location'' 
# IRRELEVANT NOW             # Mind: integer is number of MILES, NOT KM
                            # e.g. '50' is roughly equivalent to 80km
##############################################################

In [10]:
# New search URL: explicit distance and location info not taken into consideration anymore
# Instead, coded Positon and Geo info: 
# &f_E=5%2C6&geoId=104008204&  WITH f_E=5%2C6 -> Director, VP and C-Level
#                                   geoId=104008204 -> Düsseldorf
s_pos_level = 'f_E=5%2C6'             # f_E: Filter Employment type? -> only Direktoren, VP oder GL-Level
s_geo_loc = 'geoId=91000006'         # 'geoId=104008204' = Düsseldorf  # 'geoId=101282230' = Deutschland # 'geoId=91000006' = DACH  # 'geoId=103480659' = NRW 

# 3) Definitions

## 3a) Class definitions

## 3b) Function Definitions

In [11]:
def f_artificial_scrolling(search_URL):
    v_while_cycles = 0
    card_list_length = 0
    card_list_length_old = 0
    driver.get(search_URL)
    card_list = driver.find_elements(By.CLASS_NAME,'.'.join("disabled ember-view job-card-container__link job-card-list__title".split()))
    card_list_length = (len(card_list))
    print( f'CONTROL1: card_list_length: value = {card_list_length} and card_list_length_old: value = {card_list_length_old}')
    
    while (card_list_length < 25) and (v_while_cycles < 8):
        v_while_cycles += 1
        driver.execute_script("return arguments[0].scrollIntoView();", card_list[len(card_list)-1])
        card_list = driver.find_elements(By.CLASS_NAME,'.'.join("disabled ember-view job-card-container__link job-card-list__title".split()))
        card_list_length_old = card_list_length
        card_list_length = (len(card_list))
        print( f'CONTROL2: card_list_length: value = {card_list_length} and card_list_length_old: value = {card_list_length_old}')
        
    return (card_list)

In [12]:
def f_scrap_position_details():
    # Details to be scrapped from the individual clicked card aka job posting
    JobTitle = 'initial_value'
    JobTitleLink = 'initial_value'
    CompanyName = 'initial_value'
    CompanyLocation = 'initial_value'
    CompanySize = 'initial_value'
    PositionLevel = 'initial_value'
    JobDescDetails = 'initial_value'
    
    try:
     JobTitle = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__job-title").text
    except:
     JobTitle = "JobTitle not found"   
    try:
        JobTitleLink = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__content--two-pane").find_element(By.TAG_NAME,"a").get_attribute('href')
    except:
       JobTitle = "JobTitleLink not found"
    try:
        CompanyName = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__company-name").find_element(By.TAG_NAME,"a").text
    except:
        CompanyName = "CompanyName not found"
    try:
        CompanyLocation = driver.find_element(By.CLASS_NAME,"jobs-unified-top-card__bullet").text
    except:
        CompanyLocation = "CompanyLocation not found"
 
    JobDescElements = driver.find_element(By.CLASS_NAME,"jobs-description-content__text").find_elements(By.TAG_NAME,"span")
    for element in JobDescElements:
        try:
            if len(element.text) > 50:
                JobDescDetails = element.text
                break
            else:
                JobDescDetails = "These are not the JobDescDetails you were looking for"
        except:
            JobDescDetails = "JobDescDetails not found"
    
    try:
        CompanyInsights = driver.find_elements(By.CLASS_NAME,'.'.join("jobs-unified-top-card__job-insight".split()))
        PositionLevel = CompanyInsights[0].text
        CompanySize = CompanyInsights[1].text
    except:
        PositionLevel = "PostionLevel not found"
        CompanySize = "CompanySize not found"
    
    l_return = [JobTitle, JobTitleLink, CompanyName, CompanyLocation, CompanySize, PositionLevel, JobDescDetails]
    
    return (l_return)

# 4) Actual Program

## 4a) Set up connection to website with credentials

In [13]:
# Use driver to open the link
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get("https://www.linkedin.com/uas/login")
time.sleep(4)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 104.0.5112
[WDM] - Get LATEST chromedriver version for 104.0.5112 google-chrome
[WDM] - Driver [/home/ulf/.wdm/drivers/chromedriver/linux64/104.0.5112.79/chromedriver] found in cache


In [14]:
# Use login credentials to login
email=driver.find_element(By.ID, "username")
email.send_keys(USERNAME)
password=driver.find_element(By.ID, "password")
password.send_keys(PASSWORD)
time.sleep(3)
password.send_keys(Keys.RETURN)

## 4b) Getting initial information on search

In [15]:
# construct link from keyword input
s_search_pos = ''   # search string for position
for i in keywords4search:
    i2 = i.replace(' ', '%20')
    s_search_pos+=(i2)+'%20'
s_search_pos=s_search_pos[:-3]

In [16]:
s_search_pos

'Director%20Finance'

In [17]:
s_searchURL = f'https://www.linkedin.com/jobs/search/?&{s_pos_level}&{s_geo_loc}&distance={s_search_distance}&keywords={s_search_pos}&location={s_search_loc}'
print(s_searchURL)

https://www.linkedin.com/jobs/search/?&f_E=5%2C6&geoId=91000006&distance=15&keywords=Director%20Finance&location=Deutschland%2C%20Österreich%20und%20die%20Schweiz


In [18]:
driver.get(s_searchURL)

In [19]:
result = []     
link = driver.current_url
no_posts = int(driver.find_element(By.CLASS_NAME,'.'.join("display-flex t-12 t-black--light t-normal".split())).text.split(' ')[0])
print(f'For position {s_search_pos} there are {no_posts} vacancies.')

For position Director%20Finance there are 892 vacancies.


In [20]:
npages = (no_posts//25)+1
print(f'No. of pages with results: {npages}')

No. of pages with results: 36


## 4c) Main loop: loop over result pages, force "slow scrolling" thru all results and retrieve detail info from all cards

In [21]:
for i in range(0, npages*25, 25):
    print("Page:",int((i/25)+1),"of",npages)
    s_searchURL = link+"&start="+str(i)
    time.sleep(7)
    loop = 0
    card_list = f_artificial_scrolling(s_searchURL)
    print ('Length of card list on this page: '+ str(len(card_list)))
    while True:
        try:
            current_card = card_list[loop]
            loop+=1
            current_card.click() 
            time.sleep(random.randint(1,6))
            try:
                result.append(f_scrap_position_details())
            except:
                pass
        except:
            break


Page: 1 of 36
CONTROL1: card_list_length: value = 7 and card_list_length_old: value = 0
CONTROL2: card_list_length: value = 7 and card_list_length_old: value = 7
CONTROL2: card_list_length: value = 12 and card_list_length_old: value = 7
CONTROL2: card_list_length: value = 12 and card_list_length_old: value = 12
CONTROL2: card_list_length: value = 12 and card_list_length_old: value = 12
CONTROL2: card_list_length: value = 17 and card_list_length_old: value = 12
CONTROL2: card_list_length: value = 22 and card_list_length_old: value = 17
CONTROL2: card_list_length: value = 22 and card_list_length_old: value = 22
CONTROL2: card_list_length: value = 24 and card_list_length_old: value = 22
Length of card list on this page: 24
Page: 2 of 36
CONTROL1: card_list_length: value = 7 and card_list_length_old: value = 0
CONTROL2: card_list_length: value = 12 and card_list_length_old: value = 7
CONTROL2: card_list_length: value = 17 and card_list_length_old: value = 12
CONTROL2: card_list_length: val

## 4d) Bring result into data frame format and export data frame to (Excel) file

In [22]:
df = pd.DataFrame(result, columns = ["JobTitle", "JobTitleLink", "CompanyName","CompanyLocation", "CompanySize", "PositionLevel","JobDescDetails"])

In [23]:
# replace line return from html via regex
df['JobDescDetails'] = df['JobDescDetails'].replace(r'\s+|\\n', ' ', regex=True)

In [24]:
# Split long link so as to keep only the essential part, the direct linkto the job post
df[['DirectJobLink','garbage']] = df['JobTitleLink'].str.split("?", n=1, expand=True)

In [25]:
df = df.drop(['JobTitleLink', 'garbage'], axis = 1)

In [26]:
df

,JobTitle,CompanyName,CompanyLocation,CompanySize,PositionLevel,JobDescDetails,DirectJobLink
0,Controller (m/w/d),HOFMANNs,"Boxberg, Baden-Württemberg, Deutschland",1.001–5.000 Beschäftigte,Vollzeit,Wir sind ein expansives und sehr erfolgreiches...,https://www.linkedin.com/jobs/view/3238005396/
1,Controller (m/w/d),exone GmbH,"Giengen an der Brenz, Baden-Württemberg, Deuts...",51–200 Beschäftigte,Vollzeit,ARBEITEN BEI UNS Die EXTRA Computer GmbH ist e...,https://www.linkedin.com/jobs/view/3235571670/
2,Controller (m/w/d) Schwerpunkt Einkauf mit SAP...,DIS AG,"Bezirk Hamburg-Nord, Hamburg, Deutschland",5.001–10.000 Beschäftigte · HR-Services,"Vollzeit · Direktor:in, VP etc.",Diese spannende Stelle in der Großhandelsbranc...,https://www.linkedin.com/jobs/view/3311296046/
3,Controller (w/m/d),Deutsche Bahn,"Karlsruhe, Baden-Württemberg, Deutschland",10.001+ Beschäftigte · LKW-Transport,"Vollzeit · Direktor:in, VP etc.",Karriere bei der Deutschen Bahn? Einstieg über...,https://www.linkedin.com/jobs/view/3258077324/
4,Director* Finance Systems and Transformation,BioNTech SE,"Mainz, Rheinland-Pfalz, Deutschland",1.001–5.000 Beschäftigte · Biotechnologieforsc...,"Vollzeit · Direktor:in, VP etc.",Werde Mitglied der BioNTech-Familie! Als Teil ...,https://www.linkedin.com/jobs/view/3313607908/
5,Director Finance Silicon Foundry (f/m/div)*,Infineon Technologies,"München, Bayern, Deutschland",10.001+ Beschäftigte · Herstellung von Halblei...,"Vollzeit · Direktor:in, VP etc.",Would you like to influence the direction of t...,https://www.linkedin.com/jobs/view/3312528136/
6,Controller (m/w/d),GULP – experts united,"Dingolfing, Bayern, Deutschland",201–500 Beschäftigte · Personalberatung und -v...,"Vollzeit · Direktor:in, VP etc.",Einleitung Was Wir Bieten Sie suchen einen sic...,https://www.linkedin.com/jobs/view/3313090820/
7,Controller (m/w/d) Produktion,LIXIL,"Lahr/Schwarzwald, Baden-Württemberg, Deutschland",10.001+ Beschäftigte · Fertigung,"Vollzeit · Direktor:in, VP etc.","Sie möchten in einem schnell wachsenden, globa...",https://www.linkedin.com/jobs/view/3314096276/
8,CNC-Schleifer (m/w/d),Page Personnel,"Nürtingen, Baden-Württemberg, Deutschland",5.001–10.000 Beschäftigte · Personalberatung u...,"Vollzeit · Direktor:in, VP etc.",Arbeitsplatz mit neusten und modernsten Maschi...,https://www.linkedin.com/jobs/view/3314393919/
9,Group Finance Controller (w/m/d),Oxford Instruments,"Ulm, Baden-Württemberg, Deutschland",1.001–5.000 Beschäftigte · Nanotechnologiefors...,"Vollzeit · Direktor:in, VP etc.",Job-Beschreibung: Als Teil der Oxford Instrume...,https://www.linkedin.com/jobs/view/3314852853/


In [27]:
# df.to_csv("_LINKEDIN_JOB_POSTINGS.xlsx")     # no openpyxl ????
df.to_excel("_LINKEDIN_JOB_POSTINGS.xlsx")